In [1]:
import math

# === Parameters ===
N_total = 800
N_f = 16
m = 4294967296
a_lcg = 1664525
c_lcg = 1013904223
s0 = 314159265
mu = [5.0, 10.0, 3.0, 8.0, 15.0, 6.0, 12.0, 7.0, 20.0, 25.0, 30.0, 35.0, 40.0, 45.0, 50.0, 55.0]
sigma = [2.0, 3.0, 1.5, 2.5, 4.0, 2.0, 3.5, 2.0, 3.0, 4.0, 2.0, 3.0, 5.0, 2.5, 3.0, 4.0]
delta = [2.5, -3.0, 2.0, -2.0, 3.5, -1.5, 2.0, -2.5, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
default_rate = 0.30
N_sel = 6
train_frac = 0.70
T_iter = 20
lam = 0.01

# === Stage 1: Synthetic borrower generation via LCG + Box-Muller ===
state = s0
def lcg_next():
    global state
    u = state / m
    state = (a_lcg * state + c_lcg) % m
    return u

labels = [0] * N_total
features = [[0.0] * N_f for _ in range(N_total)]

for i in range(N_total):
    u_label = lcg_next()
    labels[i] = 1 if u_label < default_rate else 0
    for pair in range(0, N_f, 2):
        j1, j2 = pair, pair + 1
        u1 = lcg_next()
        u2 = lcg_next()
        u1 = max(u1, 1e-15)
        r = math.sqrt(-2.0 * math.log(u1))
        z1 = r * math.cos(2.0 * math.pi * u2)
        z2 = r * math.sin(2.0 * math.pi * u2)
        features[i][j1] = mu[j1] + delta[j1] * labels[i] + sigma[j1] * z1
        features[i][j2] = mu[j2] + delta[j2] * labels[i] + sigma[j2] * z2

# === Stage 2: Fisher discriminant feature selection ===
fisher_scores = []
for j in range(N_f):
    vals_0, vals_1 = [], []
    for i in range(N_total):
        (vals_1 if labels[i] == 1 else vals_0).append(features[i][j])
    n0, n1 = len(vals_0), len(vals_1)
    # Two-pass: mean then variance, left-to-right
    sum0 = 0.0
    for v in vals_0: sum0 += v
    mu0j = sum0 / n0
    sum1 = 0.0
    for v in vals_1: sum1 += v
    mu1j = sum1 / n1
    var0 = 0.0
    for v in vals_0: var0 += (v - mu0j) ** 2
    var0 /= n0
    var1 = 0.0
    for v in vals_1: var1 += (v - mu1j) ** 2
    var1 /= n1
    denom = var0 + var1
    if denom < 1e-15: denom = 1e-15
    fisher_scores.append((-(mu1j - mu0j) ** 2 / denom, j))

fisher_scores.sort(key=lambda x: (x[0], x[1]))
selected_indices = sorted([fisher_scores[k][1] for k in range(N_sel)])

# === Stage 3: Train/test split ===
n_train = int(math.floor(N_total * train_frac))
n_test = N_total - n_train
train_X = [[features[i][j] for j in selected_indices] for i in range(n_train)]
train_y = labels[:n_train]
test_X = [[features[i][j] for j in selected_indices] for i in range(n_train, N_total)]
test_y = labels[n_train:]

# === Stage 4: Standardization using training statistics ===
train_means = []
train_stds = []
for col in range(N_sel):
    s = 0.0
    for i in range(n_train): s += train_X[i][col]
    mean_j = s / n_train
    s2 = 0.0
    for i in range(n_train): s2 += (train_X[i][col] - mean_j) ** 2
    std_j = math.sqrt(s2 / n_train)
    if std_j < 1e-10: std_j = 1e-10
    train_means.append(mean_j)
    train_stds.append(std_j)

for i in range(n_train):
    for col in range(N_sel):
        train_X[i][col] = (train_X[i][col] - train_means[col]) / train_stds[col]
for i in range(n_test):
    for col in range(N_sel):
        test_X[i][col] = (test_X[i][col] - train_means[col]) / train_stds[col]

# === Stage 5: Logistic regression via Newton-Raphson ===
X_train = [[1.0] + train_X[i] for i in range(n_train)]
d = N_sel + 1
beta = [0.0] * d

for t in range(T_iter):
    # (a) Linear predictor
    eta = []
    for i in range(n_train):
        s = 0.0
        for j in range(d): s += X_train[i][j] * beta[j]
        eta.append(s)
    # (b) Predicted probabilities, clamped
    p = []
    for i in range(n_train):
        pi = 1.0 / (1.0 + math.exp(-eta[i]))
        p.append(max(1e-12, min(1.0 - 1e-12, pi)))
    # (c) Weights
    w = [p[i] * (1.0 - p[i]) for i in range(n_train)]
    # (d) Hessian + regularization
    H = [[0.0] * d for _ in range(d)]
    for j1 in range(d):
        for j2 in range(d):
            s = 0.0
            for i in range(n_train): s += w[i] * X_train[i][j1] * X_train[i][j2]
            H[j1][j2] = s
    for j in range(1, d): H[j][j] += lam
    # (e) Gradient + regularization
    g = [0.0] * d
    for j in range(d):
        s = 0.0
        for i in range(n_train): s += X_train[i][j] * (train_y[i] - p[i])
        g[j] = s
    for j in range(1, d): g[j] -= lam * beta[j]
    # (f) Solve H * delta = g via LU with partial pivoting (Doolittle)
    A = [row[:] for row in H]
    b = g[:]
    for k in range(d):
        max_val, max_row = abs(A[k][k]), k
        for row in range(k + 1, d):
            if abs(A[row][k]) > max_val:
                max_val = abs(A[row][k])
                max_row = row
        if max_row != k:
            A[k], A[max_row] = A[max_row], A[k]
            b[k], b[max_row] = b[max_row], b[k]
        for row in range(k + 1, d):
            factor = A[row][k] / A[k][k]
            A[row][k] = factor
            for col in range(k + 1, d): A[row][col] -= factor * A[k][col]
            b[row] -= factor * b[k]
    delta_beta = [0.0] * d
    for i in range(d - 1, -1, -1):
        s = b[i]
        for j in range(i + 1, d): s -= A[i][j] * delta_beta[j]
        delta_beta[i] = s / A[i][i]
    for j in range(d): beta[j] += delta_beta[j]

# === Stage 6: Test prediction ===
X_test = [[1.0] + test_X[i] for i in range(n_test)]
test_p = []
test_pred = []
for i in range(n_test):
    s = 0.0
    for j in range(d): s += X_test[i][j] * beta[j]
    pi = 1.0 / (1.0 + math.exp(-s))
    pi = max(1e-12, min(1.0 - 1e-12, pi))
    test_p.append(pi)
    test_pred.append(1 if pi >= 0.5 else 0)

# === Stage 7: Evaluation ===
# Log-loss
logloss = 0.0
for i in range(n_test):
    logloss += test_y[i] * math.log(test_p[i]) + (1.0 - test_y[i]) * math.log(1.0 - test_p[i])
logloss = -logloss / n_test

# Balanced accuracy
TP = FP = TN = FN = 0
for i in range(n_test):
    if test_y[i] == 1 and test_pred[i] == 1: TP += 1
    elif test_y[i] == 0 and test_pred[i] == 1: FP += 1
    elif test_y[i] == 0 and test_pred[i] == 0: TN += 1
    else: FN += 1
sensitivity = TP / (TP + FN) if (TP + FN) > 0 else 0.0
specificity = TN / (TN + FP) if (TN + FP) > 0 else 0.0
BA = (sensitivity + specificity) / 2.0

# Coefficient L2 norm
beta_norm_sq = 0.0
for j in range(d): beta_norm_sq += beta[j] ** 2
beta_norm = math.sqrt(beta_norm_sq)

# Output
T_out = 1e3 * logloss + 1e2 * (1.0 - BA) + beta_norm + N_sel
print(round(T_out, 3))

238.178


![image.png](../background_photos/)
[լուսանկարի հղումը](https://unsplash.com/photos/a-large-mountain-with-a-very-tall-cliff-UiP9KfVe3aQ), Հեղինակ՝ []()

<a href="ToDo" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a> (ToDo)

> Song reference - ToDo

# 🎦 Տեսադասեր + լրացուցիչ (ToDo)

ToDo
1. [Տեսություն 2025](https://youtu.be/TIWU_bjFuUc) \
2. [Տեսություն 2023 (ToDo)]()  \
3. [Գործնական 2025](https://youtu.be/zf8xEfdLXRo) \
4. [Գործնական 2023 (ToDo)]()  \
5. [Որոշ տնայինների լուծումներ (ToDo)]()
    
Google Forms ToDo

# 📚 Նյութը

# 🛠️ Գործնական


# 🏡Տնային


# 🎲 00
- ▶️[Video]()
- ▶️[Random link]()
- 🇦🇲🎶[]()
- 🌐🎶[]()
- 🤌[Կարգին]()


<a href="http://s01.flagcounter.com/more/1oO"><img src="https://s01.flagcounter.com/count2/1oO/bg_FFFFFF/txt_000000/border_CCCCCC/columns_2/maxflags_10/viewers_0/labels_0/pageviews_1/flags_0/percent_0/" alt="Flag Counter"></a>
